#### RNA reads processing (Illumina 2x125bp),  TPM (DNA, RNA) calculation, gene of interest annotation, and taxonomy identification

In [ ]:
# 1.Concatenate and rename files
source /nesi/project/uoa02469/Software/PipelineSetup/genomic.sh

mkdir comb-fastq
OUT=comb-fastq

for r in R1 R2;
do
    cat *4462-37*_${r}_001.fastq.gz > $OUT/RNA-S1-Sed1_${r}.fastq.gz
    cat *4462-38*_${r}_001.fastq.gz > $OUT/RNA-S1-Sed2_${r}.fastq.gz
    cat *4462-39*_${r}_001.fastq.gz > $OUT/RNA-S1-Sed3_${r}.fastq.gz
    cat *4462-41*_${r}_001.fastq.gz > $OUT/RNA-S2-Sed1_${r}.fastq.gz
    cat *4462-42*_${r}_001.fastq.gz > $OUT/RNA-S2-Sed2_${r}.fastq.gz
    cat *4462-43*_${r}_001.fastq.gz > $OUT/RNA-S2-Sed3_${r}.fastq.gz
    cat *4462-45*_${r}_001.fastq.gz > $OUT/RNA-S3-Sed1_${r}.fastq.gz
    cat *4462-46*_${r}_001.fastq.gz > $OUT/RNA-S3-Sed2_${r}.fastq.gz
    cat *4462-47*_${r}_001.fastq.gz > $OUT/RNA-S3-Sed3_${r}.fastq.gz
    cat *4462-49*_${r}_001.fastq.gz > $OUT/RNA-S4-Sed1_${r}.fastq.gz
    cat *4462-50*_${r}_001.fastq.gz > $OUT/RNA-S4-Sed2_${r}.fastq.gz
    cat *4462-51*_${r}_001.fastq.gz > $OUT/RNA-S4-Sed3_${r}.fastq.gz
    cat *4462-53*_${r}_001.fastq.gz > $OUT/RNA-S5-Sed1_${r}.fastq.gz
    cat *4462-54*_${r}_001.fastq.gz > $OUT/RNA-S5-Sed2_${r}.fastq.gz
    cat *4462-55*_${r}_001.fastq.gz > $OUT/RNA-S5-Sed3_${r}.fastq.gz
    cat *4462-57*_${r}_001.fastq.gz > $OUT/RNA-S6-Sed1_${r}.fastq.gz
    cat *4462-58*_${r}_001.fastq.gz > $OUT/RNA-S6-Sed2_${r}.fastq.gz
    cat *4462-59*_${r}_001.fastq.gz > $OUT/RNA-S6-Sed3_${r}.fastq.gz
    cat *4462-61*_${r}_001.fastq.gz > $OUT/RNA-S7-Sed1_${r}.fastq.gz
    cat *4462-62*_${r}_001.fastq.gz > $OUT/RNA-S7-Sed2_${r}.fastq.gz
    cat *4462-63*_${r}_001.fastq.gz > $OUT/RNA-S7-Sed3_${r}.fastq.gz
    cat *4462-40*_${r}_001.fastq.gz > $OUT/RNA-Filt1_${r}.fastq.gz
    cat *4462-44*_${r}_001.fastq.gz > $OUT/RNA-Filt2_${r}.fastq.gz
    cat *4462-48*_${r}_001.fastq.gz > $OUT/RNA-Filt3_${r}.fastq.gz
    cat *4462-52*_${r}_001.fastq.gz > $OUT/RNA-Filt4_${r}.fastq.gz    
    cat *4462-56*_${r}_001.fastq.gz > $OUT/RNA-Filt5_${r}.fastq.gz
    cat *4462-60*_${r}_001.fastq.gz > $OUT/RNA-Filt6_${r}.fastq.gz
    cat *4462-64*_${r}_001.fastq.gz > $OUT/RNA-Filt7_${r}.fastq.gz
    cat *4462-65*_${r}_001.fastq.gz > $OUT/RNA-Filt8_${r}.fastq.gz  
    cat *4462-66*_${r}_001.fastq.gz > $OUT/RNA-Filt9_${r}.fastq.gz   
done

mkdir fastqc/
mkdir fastqc/raw

module load FastQC/0.11.7
for i in RNA*fastq;do fastqc $i -o fastqc/raw -f fastq -t 4; done

cd fastqc/raw
module load MultiQC/1.7-gimkl-2018b-Python-3.7.3
multiqc ./ --interactive





In [ ]:
# 2. sortmerna choose only non-rRNA

mkdir 2.sortmerna

#sortmerna slurm script
#job=17566460 and 17566472

export FILES=($(ls -1 OG4462B_fastq/comb-fastq/1.comb/RNA-*_R1.fastq.gz)) && NUM=${#FILES[@]} && ZNUM=$(($NUM - 1))
if [ $ZNUM -ge 0 ]; then sbatch --array=0-$ZNUM sortmerna1.sbatch; fi
#!/bin/bash
#SBATCH -A ga02676
#SBATCH -J 1.sortrna
#SBATCH --time 8:00:00
#SBATCH --mem-per-cpu=2000
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=20
#SBATCH --requeue
#SBATCH -e slurm-sortrna.%A_%a.err
#SBATCH -o slurm-sortrna.%A_%a.out

export FILES=($(ls -1 OG4462B_fastq/comb-fastq/1.comb/RNA-*_R1.fastq.gz))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
SAMPLE=$(basename ${FILENAME} _R1.fastq.gz);
export DIR=OG4462B_fastq/comb-fastq/1.comb
    
module load BBMap/38.81-gimkl-2020a
    
reformat.sh in1=$DIR/${SAMPLE}_R1.fastq.gz in2=$DIR/${SAMPLE}_R2.fastq.gz out=$DIR/${SAMPLE}_in.fastq.gz;

gunzip $DIR/${SAMPLE}_in.fastq.gz
  
module load SortMeRNA/2.1-gimkl-2017a
sortmerna --ref sortmerna-2.1/rRNA_databases/silva-bac-16s-id90.fasta,sortmerna-2.1/index/silva-bac-16s-db:\
sortmerna-2.1/rRNA_databases/silva-bac-23s-id98.fasta,sortmerna-2.1/index/silva-bac-23s-db:\
sortmerna-2.1/rRNA_databases/silva-arc-16s-id95.fasta,sortmerna-2.1/index/silva-arc-16s-db:\
sortmerna-2.1/rRNA_databases/silva-arc-23s-id98.fasta,sortmerna-2.1/index/silva-arc-23s-db:\
sortmerna-2.1/rRNA_databases/silva-euk-18s-id95.fasta,sortmerna-2.1/index/silva-euk-18s-db:\
sortmerna-2.1/rRNA_databases/silva-euk-28s-id98.fasta,sortmerna-2.1/index/silva-euk-28s:\
sortmerna-2.1/rRNA_databases/rfam-5s-database-id98.fasta,sortmerna-2.1/index/rfam-5s-db:\
sortmerna-2.1/rRNA_databases/rfam-5.8s-database-id98.fasta,sortmerna-2.1/index/rfam-5.8s-db \
--reads $DIR/${SAMPLE}_in.fastq --num_alignments 1 --fastx --paired_in --aligned sortmerna/${SAMPLE}_rRNA --other sortmerna/mrna.${SAMPLE}_non_rRNA --log -v -a 20

#repair.sh in=sortmerna/mrna.${SAMPLE}_non_rRNA.fastq \
    out1=sortmerna/mrna.${SAMPLE}_R1.fastq out2=sortmerna/mrna.${SAMPLE}_R2.fastq \
    outs=sortmerna/mrna.${SAMPLE}_single.fastq
    
#remove empty line in sortmerna output, issue refers to https://github.com/biocore/sortmerna/issues/117
for i in sortmerna/mrna.RNA-*; 
    do export sample=$(echo ${i} | sed -e 's/sortmerna\/\(.*\)/\1/g'); 
    sed -i '/^$/d' $i ;
    done

mv sortmerna OG4462B_fastq/comb-fastq/2.sortmerna

cd OG4462B_fastq/comb-fastq/
    
for i in 2.sortmerna/*.log;do echo $i;grep 'Total' $i;done

module load FastQC/0.11.7
mkdir fastqc/mrna
    
for i in 2.sortmerna/mrna.*fastq;do fastqc $i -o fastqc/mrna -f fastq -t 4; done
    
module load MultiQC/1.7-gimkl-2018b-Python-3.7.3
cd fastqc
multiqc fastqc/mrna/ --interactive
cd ../

In [ ]:
# 3. Quality filter and trim

mkdir 3.trimmomatic
echo ">FastQC_adapter" > iua.fna
echo "AGATCGGAAGAG" >> iua.fna

#testing with iua.fna with 5:20
for i in 2.sortmerna/mrna.RNA*_R1.fastq; do  name=$(basename ${i} _R1.fastq);     
    trimmomatic PE -threads 12 -phred33 -quiet \
    2.sortmerna/${name}_R1.fastq 2.sortmerna/${name}_R2.fastq \
    3.trimmomatic/t30.q.${name}_R1.fastq.gz \
    3.trimmomatic/t30.q.${name}_R1.single1.fastq.gz \
    3.trimmomatic/t30.q.${name}_R2.fastq.gz \
    3.trimmomatic/t30.q.${name}_R2.single2.fastq.gz \
    ILLUMINACLIP:iua.fna:1:25:7   SLIDINGWINDOW:4:30 MINLEN:60; 
done

#ran fastqc again 
mkdir fastqc/trim_mrna

module load FastQC/0.11.7
for i in 3.trimmomatic/t30.*fastq.gz;do fastqc $i -o fastqc/trim_mrna -f fastq -t 4; done
cd fastqc/

module load MultiQC/1.7-gimkl-2018b-Python-3.7.3
multiqc trim_mrna/ --interactive

#but still some remaining bad bases at the end, crop last 10 bases

cd ../3.trimmomatic/

for filename in t30.q.*_R1.fastq.gz;
do
 i=$(basename ${filename} _R1.fastq.gz);

    trimmomatic PE -threads 6 -phred33 -quiet \
        ${i}_R1.fastq.gz ${i}_R2.fastq.gz \
        new.${i}_R1.fastq new.${i}_R1.single1.fastq \
        new.${i}_R2.fastq new.${i}_R2.single2.fastq \
        CROP:115 SLIDINGWINDOW:4:30 MINLEN:60

done

#final fastqc
mkdir fastqc/final-trim_mrna

module load FastQC/0.11.7
for i in 3.trimmomatic/new.*fastq;do fastqc $i -o fastqc/final-trim_mrna -f fastq -t 4; done
cd fastqc/

module load MultiQC/1.7-gimkl-2018b-Python-3.7.3
multiqc final-trim_mrna --interactive

In [ ]:
# 3. deduplication slurm script
export FILES=($(ls -1 /home/htee663/ga02676/Waiwera_project/2.Assembly_real/*sed*scaffolds.fasta))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
export SAMPLE=$(echo ${FILENAME} | sed -e 's/\/home\/htee663\/ga02676\/Waiwera_project\/2.Assembly_real\/\(.*\).scaffolds.fasta/\1/g')    

module load BBMap/37.93-gimkl-2017a

#sediments
dedupe.sh in=${FILENAME} out=dedupe.${SAMPLE}.scaffolds.fasta threads=24 minidentity=100

#water
cat *Filter.scaffolds.fasta > all.Filter.scaffolds.fasta
dedupe.sh in=all.Filter.scaffolds.fasta out=dedupe.all.Filter.scaffolds.fasta threads=24 minidentity=100

    
# 4. mapping 
#job=
mkdir 4-wts-waiwera-map-count
    
# 4.1 Water column slurm script 

TMPDIR=/nesi/nobackup/ga02676/Waiwera_project/szetest/tmp
#mkdir -p $TMPDIR
export TMPDIR

export FILES=($(ls -1 3.trimmomatic/new.t30.q.mrna.RNA-F*_R1.fastq))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
#Trimmed read directory:
export DIR1=$(echo ${FILENAME} | sed -e 's/\(3.trimmomatic\)\/.*/\1/g')
#Sample name:
export SAMPLE=$(echo ${FILENAME} | sed -e 's/3.trimmomatic\/new.t30.q.mrna.RNA-\(.*\)_R.*/\1/g')
#Contigs file name:
export CONTIGS=$(echo /home/htee663/nobackupga02676/szetest/201023_dedupe/dedupe.all.filter.fasta)
#Output directories:
export DIR2=$(echo 4-wts-waiwera-map-count)


## Map WTS read files to contigs:
module load BBMap/37.93-gimkl-2017a

srun bbmap.sh -Xmx160g in1=${DIR1}/new.t30.q.mrna.RNA-${SAMPLE}_R1.fastq \
  in2=${DIR1}/new.t30.q.mrna.RNA-${SAMPLE}_R2.fastq   \
    ref=${CONTIGS} t=36 nodisk  out=${DIR2}/${SAMPLE}.sam &> bbmap.rna.filter.${SAMPLE}.log

 grep -v '*' ${DIR2}/${SAMPLE}.sam > ${DIR2}/mapped.${SAMPLE}.sam

module load SAMtools/1.8-gimkl-2017a

srun samtools view -uS ${DIR2}/mapped.${SAMPLE}.sam | \
     samtools sort -O bam -o ${DIR2}/mapped.${SAMPLE}.bam

    
# 4.2 sediments slurm script 


TMPDIR=/nesi/nobackup/ga02676/Waiwera_project/szetest/tmp
mkdir -p $TMPDIR
export TMPDIR

export FILES=($(ls -1 3.trimmomatic/new.t30.q.mrna.RNA-S*_R1.fastq))
FILENAME=${FILES[$SLURM_ARRAY_TASK_ID]}
#Trimmed read directory:
export DIR1=$(echo ${FILENAME} | sed -e 's/\(3.trimmomatic\)\/.*/\1/g')
#Sample name:
export SAMPLE=$(echo ${FILENAME} | sed -e 's/3.trimmomatic\/new.t30.q.mrna.RNA-\(.*\)_R.*/\1/g')
export NAME=$(echo ${FILENAME} | sed -e 's/3.trimmomatic\/\(.*\)-\(Sed[1-3]\)_R.*/\2/g')
export SITE=$(echo ${FILENAME} | sed -e 's/3.trimmomatic\/new.t30.q.mrna.*RNA-\(S[1-7]\)-Sed[1-3]_R.*/\1/g')
#Output directories:
export DIR2=$(echo 4-wts-waiwera-map-count)
export DIR3=$(echo /home/htee663/nobackupga02676/szetest/201023_dedupe/)

## Map WTS read files to contigs:
module load BBMap/37.93-gimkl-2017a

srun bbmap.sh -Xmx120g in1=${DIR1}/new.t30.q.mrna.RNA-${SITE}-${NAME}_R1.fastq \
        in2=${DIR1}/new.t30.q.mrna.RNA-${SITE}-${NAME}_R2.fastq  \
    ref=${DIR3}/dedupe.${SITE}.sediment.scaffolds.fasta t=24 nodisk out=${DIR2}/${SAMPLE}.sam overwrite=true &> bbmap.rna.sed.${SITE}.log

grep -v '*' ${DIR2}/${SAMPLE}.sam > ${DIR2}/mapped.${SAMPLE}.sam

module load SAMtools/1.8-gimkl-2017a

srun samtools view -uS ${DIR2}/mapped.${SAMPLE}.sam | \
     samtools sort -O bam -o ${DIR2}/mapped.${SAMPLE}.bam


In [ ]:
# 5. hmmsearch every assembled contigs to get trusted annotation using WGS data (hmmsearch_gene_ann.sh)

# 5.1 prodigal slurm script
module load prodigal/2.6.3-GCCcore-7.4.0
for i in *fasta; do 
    export NAME=$(echo $i | sed -e 's/\.scaffolds.fasta//g'); 
    prodigal -i $i -o $NAME.gff -a $NAME.faa -d $NAME.fna -m -p met -f gff; 
    done


module load HMMER/3.2.1-gimkl-2018b

# optional: find trusted cutoff score using reference database (swissprot, seeds file from fungene)
hmmsearch --cpu  12 --tblout fam.rubisco_small.txt  ~/database/Databases/szetestdatabase/PF00101.hmm uniprot-pf00101-filtered-reviewed_yes.fasta
hmmsearch --cpu  12 --tblout fam.rubisco_large.txt  ~/database/Databases/szetestdatabase/PF00016.hmm uniprot-pf00101-filtered-reviewed_yes.fasta
hmmsearch --cpu  12 --tblout fam.amoA_comammox.txt  ~/database/Databases/szetestdatabase/amoA_comammox.hmm amoA_comammox.seeds
hmmsearch --cpu  12 --tblout fam.amoA_AOB_like.txt  ~/database/Databases/szetestdatabase/amoA_AOB_like.hmm amoA_AOB_like.seeds
hmmsearch --cpu  12 --tblout fam.amoA_AOA.txt  ~/database/Databases/szetestdatabase/amoA_AOA.hmm amoA_AOA.seeds
hmmsearch --cpu  12 --tblout fam.amoA_AOB.txt  ~/database/Databases/szetestdatabase/amoA_AOB.hmm amoA_AOB.seeds
hmmsearch --cpu  12  --tblout fam.kdpA.txt ~/database/Databases/szetestdatabase/PF03814.hmm uniprot-kdpa-filtered-reviewed_yes.fasta
hmmsearch --cpu  12   --tblout fam.kdpc.txt ~/database/Databases/szetestdatabase/PF02669.hmm uniprot-kdpc-reviewed_yes.fasta
hmmsearch --cpu  12 -T 110 --tblout fam.nhaD.txt ~/database/Databases/szetestdatabase/PF03660.hmm nhaD.fasta

# 5.2 hmmsearch against the database (refer supplementary table for genes of interested and the cutoff score)
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  63 --tblout fam.S$i.$n.mnhG.txt ~/database/Databases/szetestdatabase/TIGR01300.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  24 --tblout fam.S$i.$n.mnhF.txt ~/database/Databases/szetestdatabase/PF04066.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  21 --tblout fam.S$i.$n.mnhE.txt ~/database/Databases/szetestdatabase/PF01899.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  692 --tblout fam.S$i.$n.mnhD.txt ~/database/Databases/szetestdatabase/TIGR00944.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  162 --tblout fam.S$i.$n.mnhC.txt ~/database/Databases/szetestdatabase/TIGR00941.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  22 --tblout fam.S$i.$n.mnhB.txt ~/database/Databases/szetestdatabase/PF04039.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  920 --tblout fam.S$i.$n.mnhA.txt ~/database/Databases/szetestdatabase/TIGR00940.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  220 --tblout fam.S$i.$n.NhaA.txt ~/database/Databases/szetestdatabase/TIGR00773.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  713 --tblout fam.S$i.$n.NhaB.txt ~/database/Databases/szetestdatabase/TIGR00774.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  409 --tblout fam.S$i.$n.NhaD.txt ~/database/Databases/szetestdatabase/TIGR00775.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  26 --tblout fam.S$i.$n.Na_H_Exchanger.txt ~/database/Databases/szetestdatabase/PF00999.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  444 --tblout fam.S$i.$n.kup.txt ~/database/Databases/szetestdatabase/TIGR00794.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  407 --tblout fam.S$i.$n.bcct.txt ~/database/Databases/szetestdatabase/TIGR00842.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  21 --tblout fam.S$i.$n.proX.txt ~/database/Databases/szetestdatabase/PF04069.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  326 --tblout fam.S$i.$n.proV.txt ~/database/Databases/szetestdatabase/TIGR01186.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  27 --tblout fam.S$i.$n.TrkA-C.txt ~/database/Databases/szetestdatabase/PF02080.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  295 --tblout fam.S$i.$n.TIGR01941.txt ~/database/Databases/szetestdatabase/TIGR01941.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  31 --tblout fam.S$i.$n.PF02406.txt ~/database/Databases/szetestdatabase/PF02406.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  24 --tblout fam.S$i.$n.PsaA_PsaB.txt ~/database/Databases/szetestdatabase/PF00223.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  48 --tblout fam.S$i.$n.PsaL.txt ~/database/Databases/szetestdatabase/PF02605.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  24 --tblout fam.S$i.$n.psaF.txt ~/database/Databases/szetestdatabase/PF02507.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  21 --tblout fam.S$i.$n.psbI.txt ~/database/Databases/szetestdatabase/PF02532.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  596 --tblout fam.S$i.$n.psbA.txt ~/database/Databases/szetestdatabase/TIGR01151.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  40 --tblout fam.S$i.$n.psbZ.txt ~/database/Databases/szetestdatabase/TIGR03043.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  25 --tblout fam.S$i.$n.PII.txt ~/database/Databases/szetestdatabase/PF00421.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  606 --tblout fam.S$i.$n.bchl.txt ~/database/Databases/szetestdatabase/PF02327.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  21 --tblout fam.S$i.$n.LHC.txt ~/database/Databases/szetestdatabase/PF00556.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  24 --tblout fam.S$i.$n.cellulase.txt ~/database/Databases/szetestdatabase/PF00150.hmm S$i.$n.faa;done;done
r n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  729 --tblout fam.S$i.$n.cphA.txt ~/database/Databases/szetestdatabase/TIGR02068.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  126 --tblout fam.S$i.$n.cphB.txt ~/database/Databases/szetestdatabase/TIGR02069.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  527 --tblout fam.S$i.$n.kdpB.txt ~/database/Databases/szetestdatabase/TIGR01497.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  110 --tblout fam.S$i.$n.kdpA.txt ~/database/Databases/szetestdatabase/PF03814.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  110 --tblout fam.S$i.$n.kdpC.txt ~/database/Databases/szetestdatabase/PF02669.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  266 --tblout fam.S$i.$n.pufL.txt ~/database/Databases/szetestdatabase/TIGR01157.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12  -T  364 --tblout fam.S$i.$n.pufM.txt ~/database/Databases/szetestdatabase/TIGR01115.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12 -T 627.4 --tblout fam.S$i.$n.amoA_comammox.txt  ~/database/Databases/szetestdatabase/amoA_comammox.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12 -T 442 --tblout fam.S$i.$n.amoA_AOB_like.txt  ~/database/Databases/szetestdatabase/amoA_AOB_like.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12 -T 400 --tblout fam.S$i.$n.amoA_AOA.txt  ~/database/Databases/szetestdatabase/amoA_AOA.hmm S$i.$n.faa;done;done
for n in {sediment,Filter};do for i in {1..9};do hmmsearch --cpu  12 -T 428.4 --tblout fam.S$i.$n.amoA_AOB.txt  ~/database/Databases/szetestdatabase/amoA_AOB.hmm S$i.$n.faa;done;done

# 5.3 get the list of node name with functional genes of interested and the contig fasta sequences
module load seqtk/1.2-gimkl-2017a
for i in {1..9};do grep 'NODE' fam.S${i}.Filter* |sed  's/fam.*Filter.//g;s/.txt//g;s/:NODE/\tNODE/g' |awk '{print $2,$1,$3,$4,$5,$6,$7}'>>all.fam.filter.S$i.txt;done
for i in {1..9};do grep 'NODE' fam.S${i}.sed* |sed  's/fam.*Filter.//g;s/.txt//g;s/:NODE/\tNODE/g' |awk '{print $2,$1,$3,$4,$5,$6,$7}'>>all.fam.sed.S$i.txt;done
for i in {1..9};do grep 'NODE' all.fam.sed.S$i.txt |sed  's/ .*//g' >>list.fam.sed.S$i.txt;done
for i in {1..9};do grep 'NODE' all.fam.filter.S$i.txt |sed  's/ .*//g' >>list.fam.filter.S$i.txt;done
for i in {1..9};do seqtk subseq ../S$i.Filter.fna list.fam.filter.S$i.txt > all.fam.filter.S$i.fasta;done
for i in {1..9};do seqtk subseq ../S$i.sediment.fna list.fam.sed.S$i.txt > all.fam.sed.S$i.fasta;done
for i in {1..9};do grep 'NODE' all.fam.sed.S$i.txt |sed  's/ .*//g;s/\(cov_[0-9]*.[0-9]*\)_.*/\1/g' >>list.fam.scaffold.sed.S$i.txt;done
for i in {1..9};do grep 'NODE' all.fam.filter.S$i.txt |sed  's/ .*//g;s/\(cov_[0-9]*.[0-9]*\)_.*/\1/g' >>list.fam.scaffold.filter.S$i.txt;done
for i in list.fam.scaffold.*;do less $i|sort -u > sort.$i;done
for i in {1..9};do seqtk subseq ~/ga02676/Waiwera_project/2.Assembly_real/S$i.Filter.scaffolds.fasta sort.list.fam.scaffold.filter.S$i.txt > all.fam.scaffold.filter.S$i.fasta;done
for i in {1..9};do seqtk subseq ~/ga02676/Waiwera_project/2.Assembly_real/S$i.sediment.scaffolds.fasta sort.list.fam.scaffold.sed.S$i.txt > all.fam.scaffold.sed.S$i.fasta;done


In [ ]:
# 6. Use kraken to identify contig taxonomy 

# get all the scaffolds
cat S*sediment.scaffolds.fasta >>all.sed.scaffolds.fasta
cat S*Filter.scaffolds.fasta >>all.filter.scaffolds.fasta

# get all the ORF
cat S*sediment.faa >>all.sed.faa
cat S*Filter.faa >>all.filter.faa
cat all.filter.faa all.sed.faa >>all.scaffold.faa

#get annotated gene name and node name
grep 'NODE' fam.*.Filter.* >> fam.all.filter.list.txt
sed -i 's/fam.*Filter.//g;s/.txt//g;s/:NODE/\tNODE/g' fam.all.filter.list.txt
awk '{print $2,$1,$3,$4,$5,$6,$7}' fam.all.filter.list.txt >fam.all.gene.filter.list.txt
rm fam.all.filter.list.txt

grep 'NODE' fam.*.sediment.* >> fam.all.sed.list.txt
sed -i 's/fam.*.sediment.//g;s/.txt//g;s/:NODE/\tNODE/g' fam.all.sed.list.txt
awk '{print $2,$1,$3,$4,$5,$6,$7}' fam.all.sed.list.txt >fam.all.gene.sed.list.txt
rm fam.all.sed.list.txt

#get the fasta sequence of gene of interested
seqtk subseq ../all.sed.faa list.faa.all.sed.fam.txt > all.sed.fam.faa
seqtk subseq ../all.filter.faa list.faa.all.filter.fam.txt > all.fil.fam.faa

#get the fasta sequence of contig of interested 
seqtk subseq all.sed.scaffolds.fasta list.all.sed.fam.txt > all.sed.fam.fasta
seqtk subseq all.filter.scaffolds.fasta list.all.filter.fam.txt > all.fil.fam.fasta


# use kraken to identify taxa of the contig/gene of interest
module load Kraken2/2.0.8-beta-gimkl-2018b

kraken2 --db  ~/database/Databases/Kraken2/ --threads 16 --confidence 0.8 --report taxa_all.faa.sed.fam.txt --quick --use-mpa-style --use-name all.sed.fam.faa >taxa_all.faa.sed.fam.txt
kraken2 --db  ~/database/Databases/Kraken2/ --threads 16 --confidence 0.8 --report taxa_all.faa.fil.fam.txt --quick --use-mpa-style --use-name all.fil.fam.faa >taxa_all.faa.fil.fam.txt

kraken2 --db  ~/database/Databases/Kraken2/ --threads 16 --confidence 0.8 --report taxa_all.sed.fam.txt --quick --use-mpa-style --use-name all.sed.fam.fasta >taxa_all.sed.fam.txt
kraken2 --db  ~/database/Databases/Kraken2/ --threads 16 --confidence 0.8 --report taxa_all.fil.fam.txt --quick --use-mpa-style --use-name all.fil.fam.fasta >taxa_all.fim.fam.txt


In [ ]:
# 7. count reads with Featurecounts (subread packages)

grep '>' all.filter.faa> list.all.filter.txt
sed -i 's/>//g;s/#/\t/g' list.all.filter.txt
awk '{print $1,$1,$2,$3,$4}' list.all.filter.txt>new.list.all.filter.txt
sed -i 's/_[0-9]*//12' new.list.all.filter.txt
rm list.all.filter.txt
echo 'GeneID Chr Start End Strand Feature' >edit.new.list.all.filter.gtf
cat new.list.all.filter.txt>>edit.new.list.all.filter.gtf
sed -i 's/ /\t/g' edit.new.list.all.filter.gtf
sed -i 's/\(.*\t.*\t.*\t.*\t\)1/\1+/g' edit.new.list.all.filter.gtf
sed -i 's/\(.*\t.*\t.*\t.*\t\)-1/\1-/g' edit.new.list.all.filter.gtf
sed 's/\([+,-]\)/\1\tCDS/g' edit.new.list.all.fil.gtf >test.edit.new.list.all.fil.gtf

# sediment do individually due to overlap node name
for i in {1..9};do grep '>' dedupe.S$i.sediment.faa > list.S$i.Sed.txt;done
for i in {1..9};do sed -i 's/>//g;s/#/\t/g' list.S$i.Sed.txt;done
for i in {1..9};do awk '{print $1,$1,$2,$3,$4}' list.S${i}.Sed.txt >new.list.S${i}.sed.txt;done
for i in {1..9};do sed -i 's/_[0-9]*//12' new.list.S${i}.sed.txt;done
for i in {1..9};do echo 'GeneID Chr Start End Strand Feature' >edit.new.list.S$i.sed.gtf;done
for i in {1..9};do sed 's/_[0-9]*//12' new.list.S$i.sed.txt >>edit.new.list.S$i.sed.gtf;done
for i in {1..9};do sed -i 's/ /\t/g;s/\(.*\t.*\t.*\t.*\t\)1/\1+/g;s/\(.*\t.*\t.*\t.*\t\)-1/\1-/g' edit.new.list.S$i.sed.gtf;done
for i in {1..9};do sed 's/\([+,-]\)/\1\tCDS/g' edit.new.list.S$i.sed.gtf >test.edit.new.list.S$i.sed.gtf;done

# example format required for featurecount input gtf file 
GeneID	Chr	Start	End	Strand
497097	chr1	3204563	3207049	-
497097	chr1	3411783	3411982	-
497097	chr1	3660633	3661579	-
...

#count number of ORF in each file
for i in test.edit.new.list*;do echo $i;less $i|wc -l;done                           

#get unique features from annotation file (make sure no repeat)
for i in test.edit.new.list.S*;do less $i | awk '!a[$1]++' >01.$i;done

#RNA
ml Subread/2.0.0-GCC-9.2.0

featureCounts -g GeneID -F SAF -t CDS -a test.edit.new.list.all.filter.gtf  \
-T 12 -o 3.annotation/featurecount_wts_all_filter.txt 4-wts-waiwera-map-count/mapped.Filt*bam
awk -F "\t" '{if(($7!=0) || ($8!=0) || ($9!=0)|| ($10!=0)|| ($11!=0)|| ($12!=0)|| ($13!=0)|| ($14!=0)|| ($15!=0)) {print}}' \
    3.annotation/featurecount_wts_all_filter.txt >3.annotation/edited.featurecount_wts_all_filter.txt

for i in {1..7};do 
    featureCounts -p -g GeneID -t CDS -F SAF -a 01.test.edit.new.list.S${i}.sed.gtf \
    -T 24 -o 3.annotation/featurecount_wts_s${i}_sed.txt 4-wts-waiwera-map-count/mapped.S${i}-Sed?.bam;
    done
for i in {1..7};do 
    awk '{if($7!=0 || $8!=0 || $9!=0) print}' 3.annotation/featurecount_wts_s${i}_sed.txt \
    >3.annotation/edit.featurecount_wts_s${i}_sed.txt;done

#DNA
featureCounts -g GeneID -F SAF -t CDS -a 2-wgs-waiwera-map-count/test.edit.new.list.all.filter.gtf \
    -T 12 -o 3.annotation/featurecount_wgs_all_filter.txt 2-wgs-waiwera-map-count/mapped.Filt.S*bam
awk -F "\t" '{if(($7!=0) || ($8!=0) || ($9!=0)|| ($10!=0)|| ($11!=0)|| ($12!=0)|| ($13!=0)|| ($14!=0)|| ($15!=0)) {print}}' \
    3.annotation/featurecount_wgs_all_filter.txt >3.annotation/edited.featurecount_wgs_all_filter.txt

for i in {1..9};do 
    featureCounts -p -g GeneID -t CDS -F SAF -a 2-wgs-waiwera-map-count/01.test.edit.new.list.S${i}.sed.gtf  \
    -T 24 -o 3.annotation/featurecount_wgs_s${i}_sed.txt 2-wgs-waiwera-map-count/mapped.Sed.S${i}R?.bam;done

for i in {1..9};do 
    awk '{if($7!=0 || $8!=0 || $9!=0) print}' 3.annotation/featurecount_wgs_s${i}_sed.txt \
    >3.annotation/edit.featurecount_wgs_s${i}_sed.txt;done
    
#add site name to the sample
for f in s1 s2 s3 s4 s5 s6 s7 s8 s9; do sed "s/$/\t$f/" edit.featurecount_wgs_$f\_sed.txt >nedit.featurecount_wgs_$f\_sed.txt; done
for f in s1 s2 s3 s4 s5 s6 s7; do sed "s/$/\t$f/" edit.featurecount_wts_$f\_sed.txt >nedit.featurecount_wts_$f\_sed.txt; done
for f in edited.featurecount_w*all_filter*; do sed "s/$/\tall filter/" $f > n$f; done

for i in 1 2 3 4 5 6 7 8 9;do sed -i "1d; 0,/s${i}/s//site/" "nedit.featurecount_wgs_s${i}_sed.txt" ;done
for i in 1 2 3 4 5 6 7;do sed -i "1d; 0,/s${i}/s//site/" "nedit.featurecount_wts_s${i}_sed.txt" ;done
for i in nedited.featurecount_w*all_filter*; do sed -i "1d; 0,/all filter/s//site/" "$i" ;done

In [ ]:
# 8. TPM
# 8.1 Get mapped mRNA library size (in bash)

# saved in a file called "CountMappedReads.sh"

#! /bin/bash

ml SAMtools/1.10-GCC-9.2.0
# file header
echo "bam_file total mapped unmapped"

# loop to count reads in all files and add results to a table
for sam_file in *.sam
do
total=$(samtools view -c $sam_file)
mapped=$(samtools view -c -F 4 $sam_file)
unmapped=$(samtools view -c -f 4 $sam_file)
echo "$sam_file $total $mapped $unmapped"
done

#then run in bash
./CountMappedReads.sh
 

# 8.2 Get mapped DNA library size
#get from DNA read mapping files using the script as above

# 8.3 read in count files using R (gene_count.r)
#calculate tpm
library(plyr)
library(readr)
library("dplyr")
library("gplots")

wts_filter<-read_delim("nedited.featurecount_wts_all_filter.txt",delim="\t")
myfiles = list.files(path=".", pattern="nedit.featurecount_wts_s[1-7]_sed.txt", full.names=TRUE)
wts_sed = ldply(myfiles, read_delim, delim="\t")
wts_sed=wts_sed[, c(1:6,10,7:9,11:28)]

wgs_filter<-read_delim("nedited.featurecount_wgs_all_filter.txt",delim="\t")
myfiles = list.files(path=".", pattern="nedit.featurecount_wgs_s[1-9]_sed.txt", full.names=TRUE)
wgs_sed = ldply(myfiles, read_delim, delim="\t")
wgs_sed=wgs_sed[, c(1:6,10,7:9,11:34)]

lib.size_mappedRNA <- read_delim("mapped_mRNA_libsize.txt",delim = "\t")
lib.size_mappedDNA <- read_delim("mapped_DNA_libsize.txt",delim = "\t")


count_wts_filter<- wts_filter[,7:15]
count_wts_sed<- wts_sed[,8:28]
count_wts_filter[,1:9]<-lapply(count_wts_filter[,1:9],as.numeric)
count_wts_sed[,1:21]<-lapply(count_wts_sed[,1:21],as.numeric)

count_wgs_filter<- wgs_filter[,7:15]
count_wgs_sed<- wgs_sed[,8:34]
count_wgs_filter[,1:9]<-lapply(count_wgs_filter[,1:9],as.numeric)
count_wgs_sed[,1:27]<-lapply(count_wgs_sed[,1:27],as.numeric)


gene_fam_filter<-read_delim("fam.all.gene.filter.list.txt",delim=" ",col_names = FALSE)
gene_fam_filter<-gene_fam_filter[,c(1,2,4,5,6,7)]

gene_fam_sed<-read_delim("fam.all.gene.sed.list.txt",delim=" ",col_names = FALSE)
gene_fam_sed<-gene_fam_sed[,c(1,2,4,5,6,7)]

tpm <- function(counts, lengths) {
  rate <- counts / lengths
  rate / sum(rate) * 1e6
}


totalrate=as.data.frame(colSums(count_wts_filter/wts_filter$Length))
totalratesed=as.data.frame(colSums(count_wts_sed/wts_sed$Length,na.rm= T))
write_delim(totalrate,"totalrate_wts_fil.txt")
write_delim(totalratesed,"totalrate_wts_sed.txt")

gtotalrate=as.data.frame(colSums(count_wgs_filter/wgs_filter$Length,na.rm= T))
gtotalratesed=as.data.frame(colSums(count_wgs_sed/wgs_sed$Length,na.rm= T))
write_delim(gtotalrate,"totalrate_wgs_fil.txt")
write_delim(gtotalratesed,"totalrate_wgs_sed.txt")

tpm_wts_fil=(wts_filter[,7:15]/wts_filter$Length)/(totalrate[col(wts_filter[,7:15])])*1e6
tpm_wts_sed=(wts_sed[,8:28]/wts_sed$Length)/(totalratesed[col(wts_sed[,8:28])])*1e6

tpm_wgs_fil=(wgs_filter[,7:15]/wgs_filter$Length)/(gtotalrate[col(wgs_filter[,7:15])])*1e6
tpm_wgs_sed=(wgs_sed[,8:34]/wgs_sed$Length)/(gtotalratesed[col(wgs_sed[,8:34])])*1e6

colSums(tpm_wts_fil, na.rm= T)
colSums(tpm_wts_sed, na.rm= T)
colSums(tpm_wgs_fil, na.rm= T)
colSums(tpm_wgs_sed, na.rm= T)

tpm_wts_fil1=cbind(wts_filter[c(1:6,16)],tpm_wts_fil)
tpm_wts_sed1=cbind(wts_sed[1:7],tpm_wts_sed)
tpm_wgs_fil1=cbind(wgs_filter[c(1:6,16)],tpm_wgs_fil)
tpm_wgs_sed1=cbind(wgs_sed[1:7],tpm_wgs_sed)



combgene_wts_filter<-left_join(gene_fam_filter,tpm_wts_fil1,by=c("X1"="Geneid"))%>% filter(Length!="NA")
combgene_wts_sed<-left_join(gene_fam_sed,tpm_wts_sed1,by=c("X1"="Geneid"))%>% filter(Length!="NA")
combgene_wgs_filter<-left_join(gene_fam_filter,tpm_wgs_fil1,by=c("X1"="Geneid"))%>% filter(Length!="NA")
combgene_wgs_sed<-left_join(gene_fam_sed,tpm_wgs_sed1,by=c("X1"="Geneid"))%>% filter(Length!="NA")
write_delim(combgene_wgs_filter,"combgene_wgs_filter.txt")
write_delim(combgene_wts_filter,"combgene_wts_filter.txt")
write_delim(combgene_wgs_sed,"combgene_wgs_sed.txt")
write_delim(combgene_wts_sed,"combgene_wts_sed.txt")


#combgene_wgs_filter=read.delim(file="combgene_wgs_filter.txt", sep="\t", header=T)
#combgene_wgs_sed=read.delim(file="combgene_wgs_sed.txt", sep="\t", header=T)

#filtered out genes that are duplicated/not interested
edit_gene_wgs_filter<-combgene_wgs_filter %>% filter(X2!="NA_sulfate",X2!="GROEL",X2!="pfk-eu",X2!="pfk-prok",X2!="rubisco_small", X2!="rubisco_large",X2!="TIGR02891",X2!="TrkA-N",X2!="Na_H_Exchanger",X2!="TIGR00455",X2!="TIGR00780",X2!="TIGR03479",X2!="TIGR03478",X2!="PF01654",X2!="TIGR02416",X2!="TIGR02866",X2!="TIGR02891",X2!="TIGR00781",X2!="TIGR00782",X2!="TIGR02882",X2!="TIGR01432",X2!="PF01654",X2!="TIGR00203",X2!="TIGR01433",X2!="TIGR01473",X2!="TIGR02847",X2!="TIGR00193",X2!="TIGR01792")
edit_gene_wgs_sed<-combgene_wgs_sed %>% filter(X2!="NA_sulfate",X2!="GROEL",X2!="pfk-eu",X2!="pfk-prok",X2!="rubisco_small", X2!="rubisco_large",X2!="TIGR02891",X2!="TrkA-N",X2!="Na_H_Exchanger",X2!="TIGR00455",X2!="TIGR00780",X2!="TIGR03479",X2!="TIGR03478",X2!="PF01654",X2!="TIGR02416",X2!="TIGR02866",X2!="TIGR02891",X2!="TIGR00781",X2!="TIGR00782",X2!="TIGR02882",X2!="TIGR01432",X2!="PF01654",X2!="TIGR00203",X2!="TIGR01433",X2!="TIGR01473",X2!="TIGR02847",X2!="TIGR00193",X2!="TIGR01792")
edit_gene_wts_sed<-combgene_wts_sed %>% filter(X2!="NA_sulfate",X2!="GROEL",X2!="pfk-eu",X2!="pfk-prok",X2!="rubisco_small", X2!="rubisco_large",X2!="TIGR02891",X2!="TrkA-N",X2!="Na_H_Exchanger",X2!="TIGR00455",X2!="TIGR00780",X2!="TIGR03478",X2!="TIGR03479",X2!="PF01654",X2!="TIGR02416",X2!="TIGR02866",X2!="TIGR02891",X2!="TIGR00781",X2!="TIGR00782",X2!="TIGR02882",X2!="TIGR01432",X2!="PF01654",X2!="TIGR00203",X2!="TIGR01433",X2!="TIGR01473",X2!="TIGR02847",X2!="TIGR00193",X2!="TIGR01792")
edit_gene_wts_filter<-combgene_wts_filter %>% filter(X2!="NA_sulfate",X2!="GROEL",X2!="pfk-eu",X2!="pfk-prok",X2!="rubisco_small", X2!="rubisco_large",X2!="TIGR02891",X2!="TrkA-N",X2!="Na_H_Exchanger",X2!="TIGR00455",X2!="TIGR00780",X2!="TIGR03478",X2!="TIGR03479",X2!="PF01654",X2!="TIGR02416",X2!="TIGR02866",X2!="TIGR02891",X2!="TIGR00781",X2!="TIGR00782",X2!="TIGR02882",X2!="TIGR01432",X2!="PF01654",X2!="TIGR00203",X2!="TIGR01433",X2!="TIGR01473",X2!="TIGR02847",X2!="TIGR00193",X2!="TIGR01792")

#count no occurance
#see which node with more than 1 count
n_occur <- data.frame(table(edit_gene_wgs_filter$X1))
TEST2<-edit_gene_wgs_filter[edit_gene_wgs_filter$X1 %in% n_occur$Var1[n_occur$Freq > 1],]
edit_gene_wgs_filter<-edit_gene_wgs_filter[order(-edit_gene_wgs_filter$X7,edit_gene_wgs_filter$X1),] %>% distinct(X1,.keep_all = TRUE)

n_occur <- data.frame(table(edit_gene_wgs_sed$X1))
TEST3<-edit_gene_wgs_sed[edit_gene_wgs_sed$X1 %in% n_occur$Var1[n_occur$Freq > 1],]
edit_gene_wgs_sed<-edit_gene_wgs_sed[order(-edit_gene_wgs_sed$X7,edit_gene_wgs_sed$X1),] %>% distinct(X1,.keep_all = TRUE)

n_occur <- data.frame(table(edit_gene_wts_filter$X1))
TEST2<-edit_gene_wts_filter[edit_gene_wts_filter$X1 %in% n_occur$Var1[n_occur$Freq > 1],]
edit_gene_wts_filter<-edit_gene_wts_filter[order(-edit_gene_wts_filter$X7,edit_gene_wts_filter$X1),] %>% distinct(X1,.keep_all = TRUE)

n_occur <- data.frame(table(edit_gene_wts_sed$X1))
TEST3<-edit_gene_wts_sed[edit_gene_wts_sed$X1 %in% n_occur$Var1[n_occur$Freq > 1],]
edit_gene_wts_sed<-edit_gene_wts_sed[order(-edit_gene_wts_sed$X7,edit_gene_wts_sed$X1),] %>% distinct(X1,.keep_all = TRUE)


taxa_wts_all<-full_join(edit_gene_wts_filter,edit_gene_wts_sed,by=c("X1"="X1"))
taxa_wts_all[,22:32]<-lapply(taxa_wts_all[,22:32],as.character)
taxa_wts_all[,2:12]<-lapply(taxa_wts_all[,2:12],as.character)
v=which(is.na(taxa_wts_all$`4-wts-waiwera-map-count/mapped.Filt1.bam`)) %>% head(1)
taxa_wts_all[1:v,22:32]<-taxa_wts_all[1:v,2:12]
taxa_wts_all <- taxa_wts_all[ -c(2:12) ]
taxa_wts_all<-taxa_wts_all[, c(1,11:21,2:10,22:42)]
taxa_wts_all<- taxa_wts_all%>% filter(Length.y!="NA")
taxa_wts_all[,13:42][is.na(taxa_wts_all[,13:42])]<-0

colnames(taxa_wts_all) <-c("node","n1","n2","n3","n4","n5","n6","n7","n8","n9","n10","n11","f1","f2","f3","f4","f5","f6","f7","f8","f9","S1R1","S1R2","S1R3","S2R1","S2R2","S2R3","S3R1","S3R2","S3R3","S4R1","S4R2","S4R3","S5R1","S5R2","S5R3","S6R1","S6R2","S6R3","S7R1","S7R2","S7R3")
write_delim(taxa_wts_all,"tpm_wts_all.txt")

taxa_wgs_all<-full_join(edit_gene_wgs_filter,edit_gene_wgs_sed,by=c("X1"="X1"))
taxa_wgs_all[,22:32]<-lapply(taxa_wgs_all[,22:32],as.character)
taxa_wgs_all[,2:12]<-lapply(taxa_wgs_all[,2:12],as.character)
v=which(is.na(taxa_wgs_all$mapped.RNA_S1.Fil.bam)) %>% head(1)
taxa_wgs_all[1:v,22:32]<-taxa_wgs_all[1:v,2:12]
taxa_wgs_all <- taxa_wgs_all[ -c(2:12) ]
taxa_wgs_all<-taxa_wgs_all[, c(1,11:21,2:10,22:48)]
taxa_wgs_all<- taxa_wgs_all%>% filter(Length.y!="NA")
taxa_wgs_all[,13:48][is.na(taxa_wgs_all[,13:48])]<-0

colnames(taxa_wgs_all) <-c("node","n1","n2","n3","n4","n5","n6","n7","n8","n9","n10","n11","f1","f2","f3","f4","f5","f6","f7","f8","f9","S1R1","S1R2","S1R3","S2R1","S2R2","S2R3","S3R1","S3R2","S3R3","S4R1","S4R2","S4R3","S5R1","S5R2","S5R3","S6R1","S6R2","S6R3","S7R1","S7R2","S7R3","S8R1","S8R2","S8R3","S9R1","S9R2","S9R3")
write_delim(taxa_wgs_all,"tpm_wgs_all.txt")


remove(wts_filter,wgs_filter,wts_sed,wgs_sed,count_wts_filter,count_wgs_filter,count_wts_sed,count_wgs_sed)

save.image(file = 'tpm_wtswgs.RData')


In [ ]:
# 9. get taxonomy from all node from tpm (wgs wts) list
#as there are duplicate nodenames within sediment samples, I have to retrieve the nodename individually from each sample and get their corresponding fasta files for blast to get their taxonomy
for ((i=1; i<=7; i++)); do echo $(awk -F $'\t' -v i=$i '{if($12~i) {print $1}}' tpm_wts_all.txt) | awk 'BEGIN { OFS = "\n" } { $1=$1; print }' |sort -u  >nodelist.wts.s$i.txt ; done
for ((i=1; i<=9; i++)); do echo $(awk -F $'\t' -v i=$i '{if($12~i) {print $1}}' tpm_wgs_all.txt) | awk 'BEGIN { OFS = "\n" } { $1=$1; print }' |sort -u >nodelist.wgs.s$i.txt ; done
(awk -F $'\t'  '{if($12~"all") print $1}' tpm_wts_all.txt) | awk 'BEGIN { OFS = "\n" } { $1=$1; print }' |sort -u >nodelist.wts.allf.txt
(awk -F $'\t'  '{if($12~"all") print $1}' tpm_wgs_all.txt) | awk 'BEGIN { OFS = "\n" } { $1=$1; print }'| sort -u >nodelist.wgs.allf.txt

for i in {1..7};do comm nodelist.wts.s$i.txt nodelist.wgs.s$i.txt |sed 's/\s//g' > nodelist.wgswts.s$i.txt;done

comm nodelist.wts.allf.txt nodelist.wgs.allf.txt |sed 's/\s//g' > nodelist.wgswts.allf.txt

module load seqtk/1.2-gimkl-2017a
seqtk subseq ../../../201023_dedupe/dedupe.all.filter.fasta.faa nodelist.wgswts.allf.txt > node.filter.fasta
for i in {1..7}; do seqtk subseq ../../../201023_dedupe/dedupe.S$i.sediment.faa nodelist.wgswts.s$i.txt > node.s$i.fasta;done
for i in {8..9}; do seqtk subseq ../../../201023_dedupe/dedupe.S$i.sediment.faa nodelist.wgs.s$i.txt > node.s$i.fasta;done

#####NR DATABASE
ml DIAMOND/2.0.6-GCC-9.2.0

diamond makedb --in NCBI_20200213/nr.gz -d nr.dmnd \ 
    --taxonmap prot.accession2taxid \ 
    --taxonnodes taxdump/nodes.dmp \ 
    --taxonnames taxdump/names.dmp

for i in node.*fasta;do diamond blastp -p 30  --id 80 --query-cover 70 --taxon-exclude 77133,358574,12908,151659 \
    --sensitive --top 5 -c1 -d nr.dmnd -q $i -o $i.txt \ 
    -t /tmp/ --outfmt 102

#check how many with annotation
for i in qe.dmnd_nr*txt;do awk '$2!=0 {print $0}' $i |wc -l;done
for i in qe.dmnd_nr*txt;do awk '!seen[$1]++' $i  >uniq.$i; done
header="node\ttaxa\te-value"
for file in uniq.qe.dmnd_nr_node.*;do sed -i -e "1i\ $header" $file; done

#combine multiple file into 1
awk 'FNR==1{if (NR==1) print "filename", $0; next} {print FILENAME, $0}' uniq.qe.dmnd_nr_node.* >uniq.qe.dmnd_nr_all.txt
sed -i '1d' uniq.qe.dmnd_nr_all.txt

#get taxonomy
ml Miniconda3/4.8.3
python get_taxonomy_lineage.py -i uniq.qe.dmnd_nr_all.txt -o uniq.qe.taxa.dmnd_nr_all.txt
sed -i 's/uniq.qe.dmnd_nr_node.\(.*\).fasta.txt /\1\t/g;s/filter/all filter/g' uniq.qe.taxa.dmnd_nr_all.txt

#rename some of the taxonomy names
sed -i 's/FCB group\t//g;s/Bacteroidetes\/Chlorobi group\t//g;s/Cyanobacteria\/Melainabacteria group\t//g;s/Asgard group\t//g' uniq.qe.taxa.dmnd_nr_all.txt
sed -i 's/Terrabacteria group\t//g;s/PVC group\t//g;s/TACK group\t//g;s/Nitrospinae\/Tectomicrobia group\t//g;s/delta\/epsilon subdivisions\t//g' uniq.qe.taxa.dmnd_nr_all.txt 
sed -i 's/Candidatus //g;s/candidate division //g;s/ bacterium//g' uniq.qe.taxa.dmnd_nr_all.txt


In [ ]:

# 10. match taxonomy to tpm data using R
library("ggplot2")
library("dplyr")
library("readr")
library("tidyr")
library("stringr")

#add col name first
taxa<-read.delim2("uniq.qe.taxa.dmnd_nr_all.txt",sep="\t",header = TRUE) 
label<-read.delim2("label.txt",sep="\t")

taxa_wgs_all<-read.delim2("tpm_wgs_all.txt",sep = "\t")
taxa_wts_all<-read.delim2("tpm_wts_all.txt",sep = "\t")

taxa_wgs_all<-left_join(taxa_wgs_all,label,by=c("n1"="X1"))  
taxa_wts_all<-left_join(taxa_wts_all,label,by=c("n1"="X1"))

taxa_wgs_all$newnode=taxa_wgs_all$node
taxa_wts_all$newnode=taxa_wts_all$node

taxa_wgs_all$newnode=gsub('(cov_[0-9]*.[0-9]*)_.*', '\\1', taxa_wgs_all$newnode)
taxa_wts_all$newnode=gsub('(cov_[0-9]*.[0-9]*)_.*', '\\1', taxa_wts_all$newnode)

taxa_wgs_all<-left_join(taxa_wgs_all,taxa,by=c("node"="node", "n11"="sample"))
taxa_wts_all<-left_join(taxa_wts_all,taxa,by=c("node"="node", "n11"="sample"))

taxa_wgs_all[,54:71][is.na(taxa_wgs_all[,54:71])]<-""
taxa_wts_all[,48:65][is.na(taxa_wts_all[,48:65])]<-""

taxa_wgs_all[,54:71] <- lapply(taxa_wgs_all[,54:71], as.character)
taxa_wts_all[,48:65]<- lapply(taxa_wts_all[,48:65], as.character)

taxa_wgs_all[,54:71] <- lapply(taxa_wgs_all[,54:71], as.character)
taxa_wts_all[,48:65]<- lapply(taxa_wts_all[,48:65], as.character)
taxa_wgs_all<-taxa_wgs_all %>%mutate(t5 = case_when(t4 %in% c("unclassified Archaea","environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t5)),
                                     t3 = case_when(t3 %in% c("environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t3)),
                                     t4 = case_when(t4 %in% c("unclassified Archaea","environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t4)),
                                     t5 = case_when(t5 %in% c("unclassified Archaea","environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t5)))

taxa_wts_all<-taxa_wts_all %>%mutate(t5 = case_when(t4 %in% c("unclassified Archaea","environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t5)),
                                     t3 = case_when(t3 %in% c("environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t3)),
                                     t4 = case_when(t4 %in% c("unclassified Archaea","environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t4)),
                                     t5 = case_when(t5 %in% c("unclassified Archaea","environmental samples","unclassified Bacteria","uncultured Mediterranean phage") ~ "",TRUE ~ as.character(t5))) 

taxa_wts_all$t5<-str_replace(taxa_wts_all$t5,"unclassified*","") 
taxa_wgs_all$t5<-str_replace(taxa_wgs_all$t5,"unclassified*","") 

taxa_wgs_all$t3[taxa_wgs_all$t3==""]<-"unknown"
taxa_wts_all$t3[taxa_wts_all$t3==""]<-"unknown"

taxa_wgs_all<-taxa_wgs_all%>% unite("taxa",t3:t18,remove = FALSE) 
taxa_wts_all<-taxa_wts_all%>% unite("taxa",t3:t14,remove = FALSE) 

taxa_wgs_all<-taxa_wgs_all%>% unite("taxac",t3:t5,remove = FALSE) 
taxa_wts_all<-taxa_wts_all%>% unite("taxac",t3:t5,remove = FALSE) 

taxa_wgs_all<-taxa_wgs_all%>% unite("taxaorder",t3:t6,remove = FALSE) 
taxa_wts_all<-taxa_wts_all%>% unite("taxaorder",t3:t6,remove = FALSE) 

taxa_wgs_all<-taxa_wgs_all%>% unite("taxafam",t3:t7,remove = FALSE) 
taxa_wts_all<-taxa_wts_all%>% unite("taxafam",t3:t7,remove = FALSE) 

taxa_wgs_all<-taxa_wgs_all%>% unite("taxagen",t3:t8,remove = FALSE) 
taxa_wts_all<-taxa_wts_all%>% unite("taxagen",t3:t8,remove = FALSE) 

#filter out unwanted functions
taxa_wgs_all = taxa_wgs_all %>% filter(!str_detect(X3,"Octaheme*"))
taxa_wgs_all = taxa_wgs_all %>% filter(!str_detect(X3,"nitrile*"))
taxa_wgs_all = taxa_wgs_all %>% filter(!str_detect(X3,"calcium"))
taxa_wgs_all = taxa_wgs_all %>% filter(!str_detect(X3,"PQQGDH"))
taxa_wgs_all = taxa_wgs_all %>% filter(!str_detect(X3,"Phosphodiesterase*"))
taxa_wgs_all = taxa_wgs_all %>% filter(!str_detect(X3,"disproportionation"))
taxa_wgs_all = taxa_wgs_all %>% filter(!str_detect(X3,"urease"))

taxa_wts_all = taxa_wts_all %>% filter(!str_detect(X3,"Octaheme*"))
taxa_wts_all = taxa_wts_all %>% filter(!str_detect(X3,"nitrile*"))
taxa_wts_all = taxa_wts_all %>% filter(!str_detect(X3,"calcium"))
taxa_wts_all = taxa_wts_all %>% filter(!str_detect(X3,"PQQGDH"))
taxa_wts_all = taxa_wts_all %>% filter(!str_detect(X3,"Phosphodiesterase*"))
taxa_wts_all = taxa_wts_all %>% filter(!str_detect(X3,"disproportionation"))
taxa_wts_all = taxa_wts_all %>% filter(!str_detect(X3,"urease"))

# sulfur annotation (sequence of rdsr and dsr are highly similar, can be separated into different clusters by aligning to the reference and plot tree)
taxa_wgs_all[,13:48] <- lapply(taxa_wgs_all[,13:48], as.numeric)
taxa_wts_all[,13:42] <- lapply(taxa_wts_all[,13:42], as.numeric)

###get taxonomy node from R data
rdsrA<-(taxa_wts_all[grep("dsrA",taxa_wts_all$X4), ])
rdsrA<- mutate(rdsrA,total=rowSums(rdsrA[13:42]))
rdsrA<-rdsrA[order(rdsrA$total,decreasing = T),]
rdsrA1<-as.data.frame(rdsrA[1:1294,1])
write_delim(x = rdsrA1,"rdsrA1.wts.txt")

rdsrA<-(taxa_wgs_all[grep("dsrA",taxa_wgs_all$X4), ])
rdsrA<- mutate(rdsrA,total=rowSums(rdsrA[13:48]))
rdsrA<-rdsrA[order(rdsrA$total,decreasing = T),]
rdsrA1<-as.data.frame(rdsrA[1:2077,1])
write_delim(x = rdsrA1,"rdsrA1.wgs.txt")

rdsrB<-(taxa_wts_all[grep("dsrB",taxa_wts_all$X4), ])
rdsrB<- mutate(rdsrB,total=rowSums(rdsrB[13:42]))
rdsrB<-rdsrB[order(rdsrB$total,decreasing = T),]
rdsrB1<-as.data.frame(rdsrB[1:1351,1])
write_delim(x = rdsrB1,"rdsrB1.wts.txt")

rdsrB<-(taxa_wgs_all[grep("dsrB",taxa_wgs_all$X4), ])
rdsrB<- mutate(rdsrB,total=rowSums(rdsrB[13:48]))
rdsrB<-rdsrB[order(rdsrB$total,decreasing = T),]
rdsrB1<-as.data.frame(rdsrB[1:2176,1])
write_delim(x = rdsrB1,"rdsrB1.wgs.txt")

#then use seqtk in terminal to get sequence and download swissprot sequence as reference
cat rdsrA1.w* |sort -u > rdsrA1.txt
cat rdsrB1.w* |sort -u > rdsrB1.txt
module load seqtk/1.2-gimkl-2017a
seqtk subseq all.scaffold.faa rdsrA1.txt >rdsrA.fasta
seqtk subseq all.scaffold.faa rdsrB1.txt >rdsrB.fasta

#add reference from swissprot and uniprot
cat ref*dsrA*.fasta rdsrA.fasta > rdsrA1.fasta
cat ref*dsrB*.fasta rdsrB.fasta > rdsrB1.fasta

#then align with mafft and get ML tree with fasttree
module load MAFFT/7.310-gimkl-2017a
mafft --thread 24 --maxiterate 100 rdsrA1.fasta >align.rdsrA.fasta
mafft --thread 24 --maxiterate 100 rdsrB1.fasta >align.rdsrB.fasta

#edit header as some of them have duplicate node name from different samples
sed -i 's/\(.*_[0-9]*\) \# \([0-9]*\) \# \([0-9]*\).*/\1__\2__\3/g' align.rdsrA.fasta
sed -i 's/\(.*_[0-9]*\) \# \([0-9]*\) \# \([0-9]*\).*/\1__\2__\3/g' align.rdsrB.fasta

module load FastTree/2.1.10-gimkl-2018b
export OMP_NUM_THREADS=12
FastTree -gamma align.rdsrA.fasta > align1.rdsrA.tree
FastTree -gamma align.rdsrB.fasta > align.rdsrB.tree


#input into ITOL (https://itol.embl.de/itol.cgi), look at the tree branch/cluster, and copy node name that are with rdsr cluster
#edit the node name to match the real node name
sed -i 's/\(.*cov_[0-9]*.[0-9]*_[0-9]*\)_\([0-9]*\)_\([0-9]\)/\1\t\2\t\3/g' node.rdsrA.txt
sed -i 's/\(.*cov_[0-9]*.[0-9]*_[0-9]*\)_\([0-9]*\)_\([0-9]\)/\1\t\2\t\3/g' node.rdsrA.txt

#then run in r
taxa_wgs_all$X4 = as.character(taxa_wgs_all$X4)
taxa_wts_all$X4 = as.character(taxa_wts_all$X4)

node_rdsrA<-read.delim("node.rdsrA.txt",header = F)
node_rdsrB<-read.delim("node.rdsrB.txt",header = F)


taxa_wgs_all = taxa_wgs_all %>% mutate( X4 = case_when( node %in% node_rdsrA$V1 & n7 %in% node_rdsrA$V2~ 'rdsrA', node %in% node_rdsrB$V1 & n7 %in% node_rdsrB$V2~ 'rdsrB', TRUE ~ X4 ) )  
taxa_wts_all = taxa_wts_all %>% mutate( X4 = case_when( node %in% node_rdsrA$V1 & n7 %in% node_rdsrA$V2~ 'rdsrA', node %in% node_rdsrB$V1 & n7 %in% node_rdsrB$V2~ 'rdsrB', TRUE ~ X4 ) ) 
taxa_wgs_all = taxa_wgs_all %>% mutate( X3 = case_when( X4 == "dsrA" ~ 'Sulfite_reduction', X4 =="dsrB" ~ 'Sulfite_reduction', TRUE ~ X3 ) )
taxa_wts_all = taxa_wts_all %>% mutate( X3 = case_when( X4 == "dsrA" ~ 'Sulfite_reduction', X4 =="dsrB" ~ 'Sulfite_reduction', TRUE ~ X3 ) )

# Final writing to output
taxa_wgs_all[,13:48] <- lapply(taxa_wgs_all[,13:48], as.character)
taxa_wts_all[,13:42] <- lapply(taxa_wts_all[,13:42], as.character)

taxa_wgs_all[,13:48] <- lapply(taxa_wgs_all[,13:48], as.numeric)
taxa_wts_all[,13:42] <- lapply(taxa_wts_all[,13:42], as.numeric)


write_delim(taxa_wgs_all,"taxa_wgs_all.txt")
write_delim(taxa_wts_all,"taxa_wts_all.txt")


## manual checking for SAR group== diatoms using kraken
###get taxonomy node from R data
SAR_photo<-with(taxa_wts_all,taxa_wts_all[grepl("Sar",taxa_wts_all$t4) & grepl("Photosynthesis", taxa_wts_all$X2), ])
SAR_photo<- mutate(SAR_photo,total=rowSums(SAR_photo[13:42]))
SAR_photo<-SAR_photo[order(SAR_photo$total,decreasing = T),]
SAR_photo1<-as.data.frame(SAR_photo[1:1302,7])
write_delim(x = SAR_photo1,"sar_photosynthesis.txt")                                            
    
SAR_car<-with(taxa_wts_all,taxa_wts_all[grepl("Sar",taxa_wts_all$t4) & grepl("Carbon fixation", taxa_wts_all$X2), ])
SAR_car<- mutate(SAR_car,total=rowSums(SAR_car[13:42]))
SAR_car<-SAR_car[order(SAR_car$total,decreasing = T),]
SAR_car1<-as.data.frame(SAR_car[1:71,7])
write_delim(x = SAR_car1,"sar_carfix.txt")     
                                            
module load seqtk/1.2-gimkl-2017a
seqtk subseq all.scaffold.fasta sar_photosynthesis.txt >sar_photosynthesis.fasta                        
seqtk subseq all.scaffold.fasta sar_carfix.txt >sar_carfix.fasta 

module load Kraken2/2.0.8-beta-gimkl-2018b
kraken2 --db  ~/database/Databases/Kraken2/ --threads 16 sar_carfix.fasta --report taxa.sar.carfix.txt --use-mpa-style --use-name >taxa.sar.carfix.d.txt
kraken2 --db  ~/database/Databases/Kraken2/ --threads 16 sar_photosynthesis.fasta --report taxa.sar.photo.txt --use-mpa-style --use-name >taxa.sar.photo.d.txt
                                            
#count no occurance
n_occur <- data.frame(table(taxa_wgs_all$node))
n_occur <- data.frame(table(taxa_wts_all$node))


#select the value out
group_wgs<-taxa_wgs_all %>% select(c(X2,X3,X4,f1:S9R3))
group_wts<-taxa_wts_all %>% select(c(X2,X3,X4,f1:S7R3))
group_wgs<-group_by(group_wgs,X2,X3,X4)
group_wts<-group_by(group_wts,X2,X3,X4)


#Change to numeric
group_wgs[,4:12]<-lapply(group_wgs[,4:12],as.numeric)
group_wts[,4:12]<-lapply(group_wts[,4:12],as.numeric)


#rename column names
colnames(group_wgs)<-c("PATHWAY","FUNCTION","GENE","f1","f2","f3","f4","f5","f6","f7","f8","f9","S1R1","S1R2","S1R3","S2R1","S2R2","S2R3","S3R1","S3R2","S3R3","S4R1","S4R2","S4R3","S5R1","S5R2","S5R3","S6R1","S6R2","S6R3","S7R1","S7R2","S7R3","S8R1","S8R2","S8R3","S9R1","S9R2","S9R3")
colnames(group_wts)<-c("PATHWAY","FUNCTION","GENE","f1","f2","f3","f4","f5","f6","f7","f8","f9","S1R1","S1R2","S1R3","S2R1","S2R2","S2R3","S3R1","S3R2","S3R3","S4R1","S4R2","S4R3","S5R1","S5R2","S5R3","S6R1","S6R2","S6R3","S7R1","S7R2","S7R3")

#summarise by each column by group
group_wgs<-summarise(group_wgs,sum(f1),sum(f2),sum(f3),sum(f4),sum(f5),sum(f6),sum(f7),sum(f8),sum(f9),sum(S1R1),sum(S1R2),sum(S1R3),sum(S2R1),sum(S2R2),sum(S2R3),sum(S3R1),sum(S3R2),sum(S3R3),sum(S4R1),sum(S4R2),sum(S4R3),sum(S5R1),sum(S5R2),sum(S5R3),sum(S6R1),sum(S6R2),sum(S6R3),sum(S7R1),sum(S7R2),sum(S7R3),sum(S8R1),sum(S8R2),sum(S8R3),sum(S9R1),sum(S9R2),sum(S9R3))
group_wts<-summarise(group_wts,sum(f1),sum(f2),sum(f3),sum(f4),sum(f5),sum(f6),sum(f7),sum(f8),sum(f9),sum(S1R1),sum(S1R2),sum(S1R3),sum(S2R1),sum(S2R2),sum(S2R3),sum(S3R1),sum(S3R2),sum(S3R3),sum(S4R1),sum(S4R2),sum(S4R3),sum(S5R1),sum(S5R2),sum(S5R3),sum(S6R1),sum(S6R2),sum(S6R3),sum(S7R1),sum(S7R2),sum(S7R3))


write_csv(group_wgs,"tpm_group_wgs_all.csv")
write_csv(group_wts,"tpm_group_wts_all.csv")



# For piechart

taxa_wgs_all1=cbind((taxa_wgs_all[,1:21]),(t(apply(taxa_wgs_all[,22:48], 1, tapply, gl(9, 3), mean, na.rm = TRUE))),(taxa_wgs_all[,49:76])) %>% as.data.frame()
taxa_wts_all1=cbind((taxa_wts_all[,1:21]),t(apply(taxa_wts_all[,22:42], 1, tapply, gl(7, 3), mean, na.rm = TRUE)),(taxa_wts_all[,43:70]))  %>% as.data.frame()

group_wgs1<-taxa_wgs_all1 %>% select(c(X2,f1:'9'))
group_wts1<-taxa_wts_all1 %>% select(c(X2,f1:'7'))
group_wgs1<-group_by(group_wgs1,X2)
group_wts1<-group_by(group_wts1,X2)
#summarise by each column by group
group_wgs1<-summarise(group_wgs1,sum(f1),sum(f2),sum(f3),sum(f4),sum(f5),sum(f6),sum(f7),sum(f8),sum(f9),sum(`1`),sum(`2`),sum(`3`),sum(`4`),sum(`5`),sum(`6`),sum(`7`),sum(`8`),sum(`9`))
group_wts1<-summarise(group_wts1,sum(f1),sum(f2),sum(f3),sum(f4),sum(f5),sum(f6),sum(f7),sum(f8),sum(f9),sum(`1`),sum(`2`),sum(`3`),sum(`4`),sum(`5`),sum(`6`),sum(`7`))
write_csv(group_wgs1,"tpm_group_X2wgs_all.csv")
write_csv(group_wts1,"tpm_group_X2wts_all.csv")
